In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/usr0/home/naveenr/projects/food_rescue_feedback')

In [3]:
import numpy as np
import random 
import matplotlib.pyplot as plt
from feedback.fr_feedback import *
import feedback.secret as secret
from feedback.database import open_connection

In [7]:
start_date = "2024-04-26"
end_date = "2024-05-26"

In [ ]:
annotated_feedback = pd.read_csv('../../results/reports/labeled_feedbacks_{}_{}.csv'.format(start_date,end_date))

In [42]:
annotated_feedback

,id,donation_id,recipient_location_id,published_at,created_at,updated_at,state,user_id,rescue_group_id,distance,...,donor_id,donor_location_name,donor_name,recipient_location_name,recipient_id,recipient_name,recipient_problem,inadequate_food,donor_contact_problem,info_update
0,747878,671177,10441,2024-05-08 11:00:39.796523,2024-03-04 13:02:36.625925,2024-05-08 15:06:26.292370,8,79226.0,NaN,11.007591,...,7992,NaN,North Lime Coffee & Donuts,NaN,3972,Walnut St Com Min,True,True,False,False
1,772316,670882,8973,2024-05-07 14:00:55.942624,2024-05-01 15:34:17.239907,2024-05-08 17:12:19.175973,8,51162.0,NaN,1.028991,...,6097,Ankeny Blvd.,Starbucks,NaN,3074,Ankeny Service Center,False,False,False,False
2,768853,673725,6483,2024-05-13 10:00:33.443625,2024-04-24 15:37:31.808947,2024-05-13 12:17:22.458991,8,79400.0,NaN,0.941377,...,8217,Flowers Baking Co. Woodlawn,Flowers Baking Co.,Food for the Soul,2055,Community of the Transfiguration,False,False,False,False
3,781652,671791,10937,2024-05-08 19:42:13.124301,2024-05-08 19:41:22.060757,2024-05-09 14:26:25.741646,8,74274.0,NaN,NaN,...,4539,Entenmann's Bakery Orange,Bimbo Bakery,Orange VA Clinic,4354,Orange VA Clinic SEE VETERANS ADMIN,False,False,False,True
4,763473,670489,7854,2024-05-06 19:30:35.561459,2024-04-08 15:50:12.494235,2024-05-08 19:56:42.904400,8,75670.0,NaN,0.597759,...,6442,Mikey's Late Night Slice - Elm St,Mikey's Late Night Slice,Center for Respite Care,2430,Center for Respite Care,True,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
684,755862,671696,8280,2024-05-08 17:00:09.425422,2024-03-21 14:46:07.787154,2024-05-14 17:11:31.183339,8,79569.0,NaN,0.307579,...,6449,South Philadelphia,The Rounds,NaN,2673,Church of Pentecost,True,True,True,False
685,783373,672754,7908,2024-05-13 20:25:45.615478,2024-05-10 19:56:38.368025,2024-05-14 17:53:05.720184,8,23970.0,NaN,NaN,...,3946,West 2 Avenue,Patisserie Lebeau,NaN,2453,South Granville Seniors Centre,False,False,False,False
686,743185,666870,10145,2024-04-29 17:00:35.724800,2024-02-23 19:11:54.256831,2024-04-29 19:34:48.347621,8,76174.0,NaN,5.519019,...,7159,Grandview Highway,Real Canadian Superstore,Cliff Ave.,3748,Ryan's Rainbow Emergency Food Outreach,False,False,True,False
687,702573,665834,833,2024-04-27 14:00:42.904216,2023-12-13 17:25:37.253930,2024-04-30 14:26:03.922536,8,7690.0,NaN,6.999125,...,5509,Rocky River #3297,GetGo,NaN,660,Bountiful Basement,False,True,False,False


## Fix Info

In [9]:
report_info = pd.read_csv('../../results/reports/report_info_{}_{}.csv'.format(start_date,end_date))

In [10]:
report_info

,donor_location_id,donor_location_name,donor_id,donor_name,recipient_location_id,recipient_location_name,recipient_id,recipient_name,volunteer_rating,volunteer_comment
0,6472,Entenmann's Bakery Orange,4539,Bimbo Bakery,10937,Orange VA Clinic,4354,Orange VA Clinic SEE VETERANS ADMIN,4.0,please add Aaron's phone number in notes. thanks!
1,10182,4158 William Penn Highway,6714,Crumbl Cookies,8740,Main,2938,Western Pennsylvania Search and Rescue Develop...,4.0,We were informed that they are making cookings...
2,4618,NaN,3259,Good Shepherd Presbyterian Church,1709,NaN,1344,Precious Life Shelter Inc,1.0,Please take Good Shepherd Church off the pick ...
3,7375,PR - Last Mile Food Rescue,5086,PR - Last Mile Food Rescue,10490,InterParish Ministry - IPM Food Pantry,2060,IPM Food Pantry,4.0,I've never delivered to this pantry before and...
4,10753,South Delta Baptist Church,6343,DELTA FOOD RUNNERS,9737,South Delta Route,3095,DELTA FOOD RUNNERS,4.0,Actually 19 not 20.
...,...,...,...,...,...,...,...,...,...,...
70,123,Settler's Ridge #47,74,Giant Eagle,3284,Main,1663,Sheptytsky Arms/St. George's Close,1.0,Giant eagle says only staurday Sunday and mond...
71,11279,Paris Baguette W 4th Street,7455,Paris Baguette,5423,CAIN,1903,CAIN,3.0,The map coming up in food rescue app and using...
72,12025,210 48th Street,7940,Thoro.ai,1397,Bellefield Dwellings,1098,Bellefield Dwellings,4.0,Deanne not available to accept donation. left...
73,9351,Main Location,6236,Harvie Farms,125,NaN,70,Neighborhood Resilience Project,4.0,This donation was two carloads


In [13]:
# bad_indices = []
# for index, row in report_info.iterrows():
#     a = input(row['volunteer_comment'])
#     if a.lower() == 'n':
#         bad_indices.append(index)

In [27]:
bad_comments_report_info = set([row['volunteer_comment'] for index, row in report_info.iterrows() if index in bad_indices]) 
for index, row in annotated_feedback.iterrows():
    if row['volunteer_comment'] in bad_comments_report_info:
        row['info_update'] = False

0

## Fix Donor Side

In [28]:
report_donor = pd.read_csv('../../results/reports/report_donor_{}_{}.csv'.format(start_date,end_date))

In [32]:
report_donor

,donor_location_id,feedback_count,average_volunteer_rating,donor_id,donor_name,donor_location_name,donor_contact_problem,inadequate_food,volunteer_comments,score
0,12880,8,3.000000,5888,Salem's Market & Grill,Hill District,6,4,"['2 weeks in a row no donations ', 'No donatio...",11.000000
1,6286,7,2.714286,4402,Chabaso,Warehouse,1,7,"['only 1 today ', 'only 3 today ', 'Chabasso h...",9.285714
2,13136,7,3.285714,7180,Cincinnati Children's Hospital,CCHMC - Sodexo,3,3,"[""Everyone was very kind and helpful. The reci...",6.714286
3,7328,4,1.750000,4398,The Kroger Company,Kroger 433 - Woodlawn,2,2,"['Discouraging', 'Produce 42.4 store and 42.8 ...",6.250000
4,9430,2,1.000000,5658,El Super,24-La Puente,1,2,['only gave us a box of cookies and yogurt...'...,6.000000
...,...,...,...,...,...,...,...,...,...,...
391,11111,1,4.000000,986,Starbucks*,5th Ave Amos Hall #27117,0,0,[],0.000000
392,13070,2,4.000000,8624,Chico Gleaners,Main,0,0,[],0.000000
393,10459,2,4.000000,6880,No Frills,Dean's - Fraser Street,0,0,[],0.000000
394,13220,1,4.000000,7506,Trinity Las Americas United Methodist Church,12321 Hickman Road,0,0,[],0.000000


In [43]:
donor_contact_problem = []
a = 0
for index, row in report_donor.iterrows():
    print("Index {} out of {}".format(index,len(report_donor)))
    comments = eval(row['volunteer_comments'])
    for c in comments:
        a = input(row['donor_name'] + " "+c)
        if a.lower() == 'n':
            donor_contact_problem.append(c)

Index 0 out of 396
Index 1 out of 396
Index 2 out of 396
Index 3 out of 396
Index 4 out of 396
Index 5 out of 396
Index 6 out of 396
Index 7 out of 396
Index 8 out of 396
Index 9 out of 396
Index 10 out of 396
Index 11 out of 396
Index 12 out of 396
Index 13 out of 396
Index 14 out of 396
Index 15 out of 396
Index 16 out of 396
Index 17 out of 396
Index 18 out of 396
Index 19 out of 396
Index 20 out of 396
Index 21 out of 396
Index 22 out of 396
Index 23 out of 396
Index 24 out of 396
Index 25 out of 396
Index 26 out of 396
Index 27 out of 396
Index 28 out of 396
Index 29 out of 396
Index 30 out of 396
Index 31 out of 396
Index 32 out of 396
Index 33 out of 396
Index 34 out of 396
Index 35 out of 396
Index 36 out of 396
Index 37 out of 396
Index 38 out of 396
Index 39 out of 396
Index 40 out of 396
Index 41 out of 396
Index 42 out of 396
Index 43 out of 396
Index 44 out of 396
Index 45 out of 396
Index 46 out of 396
Index 47 out of 396
Index 48 out of 396
Index 49 out of 396
Index 50 o

In [44]:
donor_contact_problem = set(donor_contact_problem)

for index, row in annotated_feedback.iterrows():
    if row['volunteer_comment'] in donor_contact_problem:
        row['donor_contact_problem'] = False

## Fix Recipient Side

In [46]:
report_recipient = pd.read_csv('../../results/reports/report_recipient_{}_{}.csv'.format(start_date,end_date))

In [48]:
recipient_contact_problem = []
a = 0
for index, row in report_recipient.iterrows():
    print("Index {} out of {}".format(index,len(report_recipient)))
    comments = eval(row['volunteer_comments'])
    for c in comments:
        a = input(row['recipient_name'] + " "+c)
        if a.lower() == 'n':
            recipient_contact_problem.append(c)

Index 0 out of 356
Index 1 out of 356
Index 2 out of 356
Index 3 out of 356
Index 4 out of 356
Index 5 out of 356
Index 6 out of 356
Index 7 out of 356
Index 8 out of 356
Index 9 out of 356
Index 10 out of 356
Index 11 out of 356
Index 12 out of 356
Index 13 out of 356
Index 14 out of 356
Index 15 out of 356
Index 16 out of 356
Index 17 out of 356
Index 18 out of 356
Index 19 out of 356
Index 20 out of 356
Index 21 out of 356
Index 22 out of 356
Index 23 out of 356
Index 24 out of 356
Index 25 out of 356
Index 26 out of 356
Index 27 out of 356
Index 28 out of 356
Index 29 out of 356
Index 30 out of 356
Index 31 out of 356
Index 32 out of 356
Index 33 out of 356
Index 34 out of 356
Index 35 out of 356
Index 36 out of 356
Index 37 out of 356
Index 38 out of 356
Index 39 out of 356
Index 40 out of 356
Index 41 out of 356
Index 42 out of 356
Index 43 out of 356
Index 44 out of 356
Index 45 out of 356
Index 46 out of 356
Index 47 out of 356
Index 48 out of 356
Index 49 out of 356
Index 50 o

In [49]:
recipient_contact_problem = set(recipient_contact_problem)

for index, row in annotated_feedback.iterrows():
    if row['volunteer_comment'] in recipient_contact_problem:
        row['recipient_problem'] = False

In [50]:
annotated_feedback.to_csv('../../results/reports/labeled_feedbacks_{}_{}.csv'.format(start_date,end_date), index=False)